# **A Multi-Tool AI Agent to Interact with Medical Datasets and Web**

**Developed By: Md. Shoaib Ahmed**

## Install Dependencies

In [ ]:
!pip install langchain langchain-core langchain-community langgraph langchain_openai google-search-results sqlalchemy pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.0 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=948203d25cdb2454938c2825c778ddd7e3b077555290261a4db677af78f66d54
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996b

## Import Libraries

In [ ]:
import pandas as pd
import sqlite3
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.utilities import SerpAPIWrapper

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Convert CSVs to SQLite

In [ ]:
heart_df = pd.read_csv("/content/drive/MyDrive/Ostad/heart.csv")
conn = sqlite3.connect("heart_disease.db")
heart_df.to_sql("heart_disease", conn, if_exists="replace", index=False)
conn.close()

In [ ]:
cancer_df = pd.read_csv("/content/drive/MyDrive/Ostad/cancer.csv")
conn = sqlite3.connect("cancer.db")
cancer_df.to_sql("cancer_data", conn, if_exists="replace", index=False)
conn.close()

In [ ]:
diabetes_df = pd.read_csv("/content/drive/MyDrive/Ostad/diabetes.csv")
conn = sqlite3.connect("diabetes.db")
diabetes_df.to_sql("diabetes_data", conn, if_exists="replace", index=False)
conn.close()

## EDA from Converted SQLite DBs

In [ ]:
import sqlite3

# --- HEART DISEASE DB ---
conn = sqlite3.connect("heart_disease.db")
print("Heart Disease DB - First 5 rows")
display(pd.read_sql_query("SELECT * FROM heart_disease LIMIT 5;", conn))

print("Count of patients in Heart dataset:")
display(pd.read_sql_query("SELECT COUNT(*) as total_patients FROM heart_disease;", conn))

print("Average age of patients in Heart dataset:")
display(pd.read_sql_query("SELECT AVG(age) as avg_age FROM heart_disease;", conn))

conn.close()


# --- CANCER DB ---
conn = sqlite3.connect("cancer.db")
print("\nCancer DB - First 5 rows")
display(pd.read_sql_query("SELECT * FROM cancer_data LIMIT 5;", conn))

print("Count of patients in Cancer dataset:")
display(pd.read_sql_query("SELECT COUNT(*) as total_patients FROM cancer_data;", conn))

# Check for a column named 'diagnosis' or similar
cols = pd.read_sql_query("PRAGMA table_info(cancer_data);", conn)
if "diagnosis" in cols["name"].tolist():
    print("Diagnosis distribution:")
    display(pd.read_sql_query("SELECT diagnosis, COUNT(*) as count FROM cancer_data GROUP BY diagnosis;", conn))

conn.close()


# --- DIABETES DB ---
conn = sqlite3.connect("diabetes.db")
print("\nDiabetes DB - First 5 rows")
display(pd.read_sql_query("SELECT * FROM diabetes_data LIMIT 5;", conn))

print("Count of patients in Diabetes dataset:")
display(pd.read_sql_query("SELECT COUNT(*) as total_patients FROM diabetes_data;", conn))

# Example: Average glucose levels if column exists
cols = pd.read_sql_query("PRAGMA table_info(diabetes_data);", conn)
if "Glucose" in cols["name"].tolist():
    print("Average Glucose level in Diabetes dataset:")
    display(pd.read_sql_query("SELECT AVG(Glucose) as avg_glucose FROM diabetes_data;", conn))

conn.close()

Heart Disease DB - First 5 rows


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


Count of patients in Heart dataset:


,total_patients
0,1025


Average age of patients in Heart dataset:


...

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Count of patients in Diabetes dataset:


,total_patients
0,768


Average Glucose level in Diabetes dataset:


,avg_glucose
0,120.894531


## Create DB Tools

In [ ]:
@tool
def HeartDiseaseDBTool(query: str) -> str:
    """Executes an SQL query on the Heart Disease dataset and returns the result."""
    conn = sqlite3.connect("heart_disease.db")
    try:
        result = pd.read_sql_query(query, conn)
    except Exception as e:
        return f"SQL Error: {str(e)}"
    finally:
        conn.close()
    return result.to_string()

@tool
def CancerDBTool(query: str) -> str:
    """Executes an SQL query on the Cancer dataset and returns the result."""
    conn = sqlite3.connect("cancer.db")
    try:
        result = pd.read_sql_query(query, conn)
    except Exception as e:
        return f"SQL Error: {str(e)}"
    finally:
        conn.close()
    return result.to_string()

@tool
def DiabetesDBTool(query: str) -> str:
    """Executes an SQL query on the Diabetes dataset and returns the result."""
    conn = sqlite3.connect("diabetes.db")
    try:
        result = pd.read_sql_query(query, conn)
    except Exception as e:
        return f"SQL Error: {str(e)}"
    finally:
        conn.close()
    return result.to_string()

## Web Search Tool (**SerpAPI**)

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = "48d5cd_Your_SERP_API_KEY" #Generate from SERP Website

MedicalWebSearchTool = SerpAPIWrapper()

## Setup Main **Agent**

In [ ]:
import os
from langchain_openai import ChatOpenAI


token = "ghp_Your_Github_Token_OPENAI" #Generate from Github Models
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4.1-mini"

if not token:
    raise ValueError("GITHUB_TOKEN environment variable not set. Please provide a valid token.")

llm =ChatOpenAI(
    model_name=model_name,
    openai_api_key=token,
    openai_api_base=endpoint,
    temperature=0.5,
)

## Create **memory** + **agent**

In [ ]:
from langchain.tools import Tool

memory = MemorySaver()
MedicalWebSearchTool = Tool(
    name="MedicalWebSearch",
    description="A tool for performing medical web searches.",
    func=MedicalWebSearchTool.run,
)
tools = [HeartDiseaseDBTool, CancerDBTool, DiabetesDBTool, MedicalWebSearchTool]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

## Run Example **Queries**

### Example 1:

In [ ]:
config = {"configurable": {"thread_id": "med_agent"}}

# Example 1: Query Heart dataset
input_message = {
    "role": "user",
    "content": "Show me the average age of patients in the heart dataset"
}
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Show me the average age of patients in the heart dataset
================================== Ai Message ==================================
Tool Calls:
  HeartDiseaseDBTool (call_RjKHNi0SFCEgrcnrKS4cNOG8)
 Call ID: call_RjKHNi0SFCEgrcnrKS4cNOG8
  Args:
    query: SELECT AVG(age) AS average_age FROM heart_disease
================================= Tool Message =================================
Name: HeartDiseaseDBTool

   average_age
0    54.434146
================================== Ai Message ==================================

The average age of patients in the heart disease dataset is approximately 54.43 years.


### Example 2:

In [ ]:
input_message = {
    "role": "user",
    "content": "Which age has the highest AlcoholIntake in the cancer dataset?"
}
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which age has the highest AlcoholIntake in the cancer dataset?
================================== Ai Message ==================================
Tool Calls:
  CancerDBTool (call_YsUCVhomnIl6DvxZ8sLO2fwb)
 Call ID: call_YsUCVhomnIl6DvxZ8sLO2fwb
  Args:
    query: SELECT age, AVG(AlcoholIntake) AS avg_alcohol_intake FROM cancer_data GROUP BY age ORDER BY avg_alcohol_intake DESC LIMIT 1
================================= Tool Message =================================
Name: CancerDBTool

   Age  avg_alcohol_intake
0   60            3.190435
================================== Ai Message ==================================

The age with the highest average AlcoholIntake in the cancer dataset is 60 years old, with an average intake of approximately 3.19.


### Example 3:

In [ ]:
input_message = {
    "role": "user",
    "content": "What are common symptoms of type 2 diabetes?"
}
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are common symptoms of type 2 diabetes?
================================== Ai Message ==================================
Tool Calls:
  MedicalWebSearch (call_Kd5DaZ8mo9Qcd5cAsxpSLoWQ)
 Call ID: call_Kd5DaZ8mo9Qcd5cAsxpSLoWQ
  Args:
    __arg1: common symptoms of type 2 diabetes
================================= Tool Message =================================
Name: MedicalWebSearch

['Type 2 diabetes type: Also called: adult onset diabetes.', 'Type 2 diabetes entity_type: medicine, health.', 'Type 2 diabetes main_tab_text: Overview.', 'Type 2 diabetes kgmid: /m/0146bp.', 'Symptoms · More thirst. · More urination. · More hunger. · Weight loss. · Tiredness. · Blurred vision. · Slow-healing sores. · Frequent infections.', 'Symptoms include increased thirst, frequent urination, fatigue, increased hunger, slow healing and Symptoms of Type 2 diabetes tend to develop slowly over time.', 'Urinating often · Fee

# Readme

##  Project Overview
This project demonstrates how to build a **Multi-Tool AI Agent** using **LangChain**, **OpenAI models**, and **SerpAPI** for medical dataset analysis and web-based queries.  
The agent can:
- Query structured **SQLite medical databases** (Heart Disease, Cancer, Diabetes).  
- Perform basic **Exploratory Data Analysis (EDA)**.  
- Search the web for real-time medical information.  
- Remember past interactions with conversational memory.  

---

##  Technologies Used
- **Python** (Google Colab)  
- **LangChain** for agent orchestration  
- **LangGraph** for memory management  
- **OpenAI (ChatOpenAI)** for reasoning and responses  
- **SerpAPI** for external medical search  
- **Pandas, NumPy, Matplotlib, Seaborn** for data handling and visualization  
- **SQLite** for storing medical datasets  

---

##  Agent Functionality
The AI agent integrates multiple tools:
- **Database Tools** → Query Heart, Cancer, and Diabetes datasets.  
- **Web Search Tool** → Fetch latest medical knowledge using SerpAPI.  
- **Custom Math Tools** → Perform simple operations like addition and multiplication.  
- **Memory** → Retains user’s past queries and answers for contextual conversation.